In [ ]:
from google.colab import files

d=files.upload()


Saving christchatbot.json to christchatbot.json


In [ ]:
import pandas as pd

df=pd.read_json('christchatbot.json')

In [ ]:
import numpy as np
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()

def tokenize(sentence):
  return nltk.word_tokenize(sentence)

def stem(word):
  return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
  tokenized_sentence=[stem(w) for w in tokenized_sentence]
  # this line will stem the words to their root form
  bag=np.zeros(len(all_words), dtype=np.float32)
  # in the above line we are creating a vector which is filled by zero
  # the length of vector is equal to number of words in all_words
  for idx , w in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx]=1.0
  # if the current word is in the tokenized sentence , then the corressponding feature of bag of words is 1.0


  return bag


sentence=["hello", "how", "are", "you"]
words=["hi", "hello","I", "you","bye","thank","cool"]
bag=bag_of_words(sentence, words)
print(bag)

a="how are you "
a=tokenize(a)
words=['organize','organizes', 'organizing ']
stemmed_words=[stem(x) for x in words]
print(stemmed_words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[0. 1. 0. 1. 0. 0. 0.]
['organ', 'organ', 'organizing ']


In [ ]:
print(df)
import torch
# torch refers to the pytorch library
# it is developed by facebook ai research lab(fair)
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

                                              intents
0   {'tag': 'greeting', 'patterns': ['Hi', 'Hey', ...
1   {'tag': 'blocks', 'patterns': ['how many build...
2   {'tag': 'hostel', 'patterns': ['do you have ho...
3   {'tag': 'goodbye', 'patterns': ['Bye', 'See yo...
4   {'tag': 'thanks', 'patterns': ['Thanks', 'Than...
5   {'tag': 'items', 'patterns': ['Which items do ...
6   {'tag': 'food', 'patterns': ['I am hungry', 'w...
7   {'tag': 'central block', 'patterns': ['i am st...
8   {'tag': 'payments', 'patterns': ['Do you take ...
9   {'tag': 'delivery', 'patterns': ['How long doe...
10  {'tag': 'funny', 'patterns': ['Tell me a joke!...


In [ ]:
import json
# we are using a context manager "with"
# context manaager allow us to read the json file in read only mode
# it allow us to close the file even when an error occurs
with open('christchatbot.json','r') as f:
  intents=json.load(f)

In [ ]:
#intent is like a goal or category, it is the main thing a person is trying to do when they talk to chatbot
all_words=[]
tags=[]
xy=[]
for intent in intents['intents']:
  tag=intent['tag']
  # here we are trying to extent the tag or label of that intent
  tags.append(tag)
  for pattern in intent['patterns']:
    # we are performing tokenize operation on all sentence of pattern
    # the result is stored in w variable
    w=tokenize(pattern)

    all_words.extend(w)
    xy.append((w,tag))
    # here we are storing words with their corresponding intent tags

ignore_words=['?','!','.',',']
all_words=[stem(w) for w in all_words if w not in ignore_words]
all_words=sorted(set(all_words))
# in the above code we are sorting the variable based on alphabetical order and removing duplicates

tags=sorted(set(tags))
print(tags)
print(all_words)
print(xy)

['blocks', 'central block', 'delivery', 'food', 'funny', 'goodbye', 'greeting', 'hostel', 'items', 'payments', 'thanks']
["'s", '1', '4', '5', 'a', 'accept', 'am', 'and', 'ani', 'anyon', 'are', 'block', 'boy', 'build', 'bye', 'can', 'canteen', 'card', 'cash', 'central', 'credit', 'day', 'deliveri', 'do', 'doe', 'eat', 'food', 'for', 'front', 'funni', 'gate', 'get', 'girl', 'go', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'hostel', 'how', 'hungri', 'i', 'in', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mani', 'mastercard', 'me', 'my', 'name', 'nearest', 'no', 'of', 'onli', 'option', 'pay', 'paypal', 'see', 'sell', 'ship', 'snack', 'someth', 'stand', 'take', 'tell', 'thank', 'that', 'the', 'there', 'to', 'want', 'what', 'when', 'where', 'which', 'with', 'you', 'your']
[(['Hi'], 'greeting'), (['Hey'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['how

In [ ]:
# code to create training dataset for a chatbot using bag of words
x_train=[]
y_train=[]
for (pattern_sentence, tag) in xy:
  bag=bag_of_words(pattern_sentence, all_words)
  # print("this is the bad",bag)
  x_train.append(bag)

  label=tags.index(tag)

  # print("this is the label",label)
  y_train.append(label)

x_train=np.array(x_train)


# vocabulary is:
# ["cat", "dog", "bird"]
#  pattern sentence is:
# "The cat sat on the mat."
#  bag-of-words representation of this pattern sentence would be the following vector:
# [1, 0, 0]

In [ ]:
# Dataset is a predeined class in Pytorch used for creating custom datasets for machine learning tasks
# chatdataset is a custom clsas that inherits pytorch dataset class
class ChatDataset(Dataset):
  # here we have implemented three methods of dataset class which are
  # a) __init__
  # b) __getitem__
  # c) __len__
  def __init__(self):
    self.n_samples=len(x_train)
    self.x_data=x_train
    self.y_data=y_train


  def __getitem__(self, index):
    # this method will return the data and it's label based on index
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    # return total number of sample in data
    return self.n_samples


batch_size=8
# model will receive 8 samples at a time to update its weights
dataset=ChatDataset()
# creating instance of chatdataset class
train_loader=DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=2)
# data loader is a utility function that help us to efficiently load and manager out data for trainig  machine learning models/deep learming model
# it is part of torch.utils.data
# shuffle is true allow data loader to shuffle the datatset at the beginning of each dataset
# num_workers will control how many worker processes are used to load data in parallel during training

In [ ]:
import torch
import torch.nn as nn

# in this network we have used three layers

class NeuralNet(nn.Module):
  # NeuralNet is a custom class
  # this will inherit a class called nn.Module
  # nn.Module is a base class for all Pytorch neural network
  def __init__(self, input_size, hidden_size, num_classes):
    # the above function is a constructor which take three parameters
    # input size - no of input features
    # hidden size is no of neurons for hidden layers
    # output layer is no of output classses
    super(NeuralNet, self).__init__()
    # network architecture
    self.l1=nn.Linear(input_size, hidden_size)
    self.l2=nn.Linear(hidden_size, hidden_size)
    self.l3=nn.Linear(hidden_size, num_classes)
     # nn.Linear - fullly connected linear layers. they specify the input and output of each layer
    self.relu=nn.ReLU()
    # rectified linear unit activation function applied after each linear layer

  def forward(self, x):
    # this method will define the forward pass of the neural network /how data flows
    out=self.l1(x) #applied first linear layer to the input x
    out=self.relu(out) # apply relu to the output of first layer
    out=self.l2(out) #apply second linear layer to the output of first layer
    out=self.relu(out)
    out=self.l3(out)

    # final out represents output of the neural network
    return out

In [ ]:
batch_size=8
hidden_size=8
output_size=len(tags)
# output is number of tags in our dataset
input_size=len(x_train[0])
learning_rate=0.001
# learning rate is a hyper parameter
# it determine how quickly or slowly a model learns
# slower model can lead to stable convergence
num_epochs=1000



print(input_size, len(all_words))
# important to check the dimesnion of input with vocabulary
print(output_size, tag)


87 87
11 funny


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# the above line checks if GPU is avaialble or not
# if yes it will use it for training
# if no cpu will be used
model=NeuralNet(input_size, hidden_size, output_size).to(device)
# creating an instance of Neuralnet custom class
criterion=nn.CrossEntropyLoss()
# defining loss function which is cross entropy loss
# it is mainly used for multi class classification problem
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)
# adam optimizer is used to update the model parameter during traiining

for epoch in range(num_epochs):
  for (words, labels) in train_loader:
    words=words.to(device)
    labels=labels.to(device)
    # the above two lines will move both input and output to device(gpu/cpu)


    outputs=model(words)
    # in above line we will pass the input to the neural network to obtain predictions
    loss=criterion(outputs, labels)


    optimizer.zero_grad()
    # the above line clear the gradient before computing new gradients during backpropagation
    loss.backward()
    # above line perform back propagation by computing gradient of loss with respect
    # to the model's parameter
    optimizer.step()
    # the above line uses optimizer to update the model's parameter (weight and bias)


  if(epoch+1)%100==0:
     print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    #  we display all epcohs whose value is multiple of 100 to track the progress



print(f'final loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 1.3713
Epoch [200/1000], Loss: 0.2902
Epoch [300/1000], Loss: 0.1844
Epoch [400/1000], Loss: 0.0108
Epoch [500/1000], Loss: 0.0052
Epoch [600/1000], Loss: 0.0070
Epoch [700/1000], Loss: 0.0012
Epoch [800/1000], Loss: 0.0014
Epoch [900/1000], Loss: 0.0002
Epoch [1000/1000], Loss: 0.0009
final loss: 0.0009


In [ ]:
data={
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags


    }

# created a python dictionary named data
# model state represent the trained weights and bias of model
# the hyperparamters (input , output and hiddent size)
# all words represent the vocabulary
# tags represent the labels in dataset

FILE="data.pth"
torch.save(data, FILE)
# the variable is stored in data.pth

print(f'training complete . file saved to {FILE}')


training complete . file saved to data.pth


In [ ]:
import random
import json

import torch




device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('christchatbot.json', 'r') as json_data:
    intents = json.load(json_data)


FILE="data.pth"
data=torch.load(FILE)
# loading pre trained neural network

input_size=data["input_size"]
hidden_size=data["hidden_size"]
output_size=data["output_size"]
all_words=data["all_words"]
tags=data["tags"]
model_state=data["model_state"]



model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
# the above line initialize weight and bias from pre trained model

model.eval()
# putting model to evaluation mode

NeuralNet(
  (l1): Linear(in_features=87, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=11, bias=True)
  (relu): ReLU()
)

In [ ]:
bot_name = "Sam"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    # the above line reshapw the bag of words to format expected by model
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: quit


In [ ]:
import pickle
# Save the trained model to a file using pickle
with open('chatbot_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)


In [ ]:
import torch
import pickle

# Load the model from the pickle file
with open('chatbot_model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

# Assuming you have the necessary functions for tokenization, bag_of_words, etc.

# Function to predict using the loaded model
def predict(sentence):
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = loaded_model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                return random.choice(intent['responses'])
    else:
        return "I do not understand..."



bot_name = "Sam"
print(f"Bot: Hi! I'm {bot_name}. Type 'quit' to exit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        print(f"Bot: Goodbye! Have a great day!")
        break

    response = predict(user_input)
    print(f"Bot: {response}")

Bot: Hi! I'm Sam. Type 'quit' to exit.
You: quit
Bot: Goodbye! Have a great day!


In [ ]:
import torch

# Assuming your loaded_model is on cuda:0
loaded_model.to('cpu')

# Create a dummy input tensor on the same device (CPU in this case)
input_tensor = torch.zeros(1, input_size).to('cpu')

# Convert the model to TorchScript
torchscript_model = torch.jit.trace(loaded_model, input_tensor)
torchscript_model.save('chatbot_model.pt')
